<a href="https://colab.research.google.com/github/cawllol/Deep-learning/blob/master/Predict_customer_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('/content/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data.shape

(10000, 14)

In [6]:
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [7]:
#Count all NaN in a DataFrame (both columns & Rows)
data.isnull().sum().sum()

0

In [8]:
X=data.iloc[:,3:-1].values
y=data.iloc[:,-1].values

In [9]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [10]:
print(y)

[1 0 1 ... 1 1 0]


In [11]:
#Label encoding the Gender column
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])

In [12]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [13]:
#One hot encoding for Geography column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [14]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [15]:
#train & test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:
#feature scalling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [17]:
#initializing ANN
ann=tf.keras.models.Sequential()

In [18]:
#adding input layer & irst hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [19]:
#output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [20]:
#compiling
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
#train the model
ann.fit(X_train,y_train,batch_size=32,epochs=100)
#ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6534 - accuracy: 0.6210
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4647 - accuracy: 0.8055
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4303 - accuracy: 0.8159
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4103 - accuracy: 0.8205
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3929 - accuracy: 0.8320
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3805 - accuracy: 0.8376
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3716 - accuracy: 0.8424
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3655 - accuracy: 0.8434
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3606 - accuracy: 0.8484
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3571 - accura

In [22]:
#Predicting the result of a single observation
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


In [23]:
#Predicting the Test set results
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [24]:
#Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1535   60]
 [ 207  198]]


0.8665